In [1]:
timestamp = "2023-03-21 22:00" # Enter timestamp in UTC in this format
api_key = "xxxxxxx-yyyy-zzzz-aaaa-bbbbbbbbbbbb" # Enter your API key
collection_contract = "0xFeffc0E5C9575576C1922978102afa2D803Dc93F" # Enter collection contract

In [2]:
# format for api_key
# api_key = "xxxxxxxx-yyyy-zzzz-aaaa-bbbbbbbbbbbb"

In [ ]:
pip install shroomdk

In [3]:
from shroomdk import ShroomDK
# Initialize `ShroomDK` with your API Key
sdk = ShroomDK(api_key)
collection_contract = collection_contract.lower()

# First query is to get a block number for snapshot that corresponds to snapshot time
sql_blocknumber = f"""
    SELECT
        block_timestamp, 
        block_number
    FROM ethereum.core.ez_nft_transfers
    WHERE date_trunc('minute', block_timestamp) = '{timestamp}'
    ORDER BY block_timestamp DESC
    LIMIT 1
"""

query_result_timestamp = sdk.query(sql_blocknumber)

for record in query_result_timestamp.records:
    snapshot = record['block_number']

In [4]:
# Parameters can be passed into SQL statements 
# via native string interpolation
# Second query is to obtain token wallet balances during a particular block 

sql = f"""
    WITH wallets as(
    SELECT 
        nft_to_address as wallet, 
        max(block_number) as last_transfer 
    FROM ethereum.core.ez_nft_transfers 
    WHERE nft_address = LOWER('{collection_contract}')
    GROUP BY wallet
    ),

    first_data as(
    SELECT
        nft_to_address as owner,
        tokenid
    FROM ethereum.core.ez_nft_transfers
    WHERE nft_address = LOWER('{collection_contract}')
    AND block_number <= '{snapshot}'
    QUALIFY RANK() OVER (
        PARTITION BY tokenid 
        ORDER BY block_number DESC, event_index DESC) = 1        
    )
    SELECT *
    FROM first_data
    ORDER BY owner
    
"""

# Run the query against Flipside's query engine 
# and await the results
query_result_set = sdk.query(sql)

In [5]:
import pandas as pd

# create empty data frame container
data = []

# convert sql run result into pandas dataframe
for record in query_result_set.records:
    owner = record['owner']
    tokenid = record['tokenid']
    data.append({'owner': owner, 'token_id': tokenid})
df = pd.DataFrame(data)

# convert token id data type from object/string to integer
df['token_id'] = df['token_id'].astype(int)

In [6]:
# group token quantity based on holder wallet
grouped_df = df.groupby(['owner']).size()

In [7]:
# create file csv file name
csv_name = 'owners_snapshot_block_' + str(snapshot) + '_collection_' + str(collection_contract) + '.csv'

# export dataframe to csv file 

grouped_df.to_csv(csv_name, index = True)